In [ ]:
#Step 7: Live trading bot

In [ ]:
# live_trading_bot.py
# Simulated live trading deployment (Binance live interface, can connect to Testnet)
# Warning: Do not use unreviewed strategies on the mainnet

import time
import numpy as np
import pandas as pd
from binance.client import Client
from binance.enums import *
import torch

# Set account information (using Binance Testnet) 
API_KEY = "YOUR_API_KEY"
API_SECRET = "YOUR_API_SECRET"
USE_TESTNET = True

# Initialize Binance client 
if USE_TESTNET:
    binance_url = 'https://testnet.binance.vision'
    client = Client(API_KEY, API_SECRET)
    client.API_URL = binance_url
else:
    client = Client(API_KEY, API_SECRET)

# Load trained PPO model 
from stable_baselines3 import PPO
from rl_trading_agent import TradingEnv
from stable_baselines3.common.vec_env import DummyVecEnv

# Prepare your recent historical data
df = pd.read_csv("cleaned_crypto_data_with_lstm_signal.csv")[-1000:].reset_index(drop=True)

# Create the environment
env = DummyVecEnv([lambda: TradingEnv(df)])
model = PPO.load("ppo_trading_agent_with_signal.zip")

# Simulated live trading function (called every 30s) 
def run_live_trading(symbol="BTCUSDT", interval=30):
    obs = env.reset()
    while True:
        try:
            # Predict position signal with the model
            action, _ = model.predict(obs, deterministic=True)
            position = float(np.clip(action[0], 0.0, 1.0))

            # Get current price
            ticker = client.get_symbol_ticker(symbol=symbol)
            price = float(ticker['price'])

            # Assume $100 USDT position (you can expand this to dynamically adjust based on balance)
            usdt_allocation = 100.0
            qty = round((position * usdt_allocation) / price, 6)

            # Simulate order (print only)
            print(f" Current Price: {price:.2f} | Position Signal: {position:.2f} | Order Quantity: {qty:.6f}")
            # order = client.order_market_buy(symbol=symbol, quantity=qty)  # Actual order (use with caution)

            obs, _, done, _ = env.step([position])
            if done[0]:
                obs = env.reset()

            time.sleep(interval)
        except Exception as e:
            print(f" Error: {e}")
            time.sleep(interval)

if __name__ == "__main__":
    print(" Launching PPO Live Trading Bot ...")
    run_live_trading()
